<a href="https://colab.research.google.com/github/mohammadreza-sheykhmousa/Geo-tools/blob/main/NLCD_compare_with_CDL_and_Landsat_timeseries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Comparing USGS, National Land Cover Database (NLCD) with USDA Cropland Data Layer (CDL) and Landsat timeseries


In [ ]:
# Install geemap
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.7 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11135 sha256=4d1fb3a3aa5383c455611e17461de9f72e28c64ffcfe164ef4f846

In [ ]:
# Import nessary packages 
import ee
import geemap

In [ ]:
# add baseamp
# Note: Earth Engine will notify you to authorize access from a google account. More info here: https://developers.google.com/earth-engine/guides/access 
Map = geemap.Map(center=[40,-100], zoom=4)
Map.add_basemap('HYBRID')
Map

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=QzFln3cp_HFvMgKp-aS35u_IripxvyfbJvUVvehnbdo&tc=qBwya8Ay97-cmD1rsT_5ygqdY8upP1jqAVWSK5GmEe8&cc=7ZXKb5wlWhwUap6-E8jfrSqIf9tTDI8Hh8Rh0yLe1Lo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk4RwGpjpork4eD_7P1oOphTEwakxV3N7h5oFS-2-LmBy_fJG-pJzOw

Successfully saved authorization token.


Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [ ]:
# Import the NLCD collection
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')

# Filter collection
nlcd2019 = dataset.filter(ee.Filter.eq('system:index', '2019')).first()

# Select land cover band
landcover = nlcd2019.select('landcover')

# Display
Map.addLayer(landcover, {}, 'NLCD 2019')
Map

Map(bottom=1851.0, center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=H…

In [ ]:
# Add title for legend and add legend to map using built in NLCD legend from geemap
title = 'NLCD Land Cover Classification'
Map.add_legend(title=title, builtin_legend='NLCD')

In [ ]:
# indate years to get images of 
dataset.aggregate_array("system:id").getInfo()
years = [ '2008', '2011', '2013', '2016', '2019']

In [ ]:
# Get an NLCD image by year
def getNLCD(year): # Import the NLCD collection.
    dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')

    # Filter the collection by year.
    nlcd = dataset.filter(ee.Filter.eq('system:index', year)).first()

    # Select the land cover band.
    landcover = nlcd.select('landcover')
    return landcover

# Get an CDL image by year
def getCDL(year): # Import the CDL collection.
    dataset = ee.ImageCollection("USDA/NASS/CDL")

    # Filter the collection by year.
    cdl = dataset.filter(ee.Filter.eq('system:index', year)).first()

    # Select the cropland band.
    cropland = cdl.select('cropland')
    return cropland
  

In [ ]:
# Create an NLCD image collection for the selected years
collection_nlcd = ee.ImageCollection(ee.List(years).map(lambda year: getNLCD(year)))

# Create an CDL image collection for the selected years
collection_cdl = ee.ImageCollection(ee.List(years).map(lambda year: getCDL(year)))

In [ ]:
# create dataset-year labels to add to timeseries inspect
labels_nlcd = [f'NLCD {year}' for year in years]
labels_cdl = [f'CDL {year}' for year in years]
labels_nlcd

['NLCD 2008', 'NLCD 2011', 'NLCD 2013', 'NLCD 2016', 'NLCD 2019']

In [ ]:
# Create timeseries inspector/slider of NLCD vs CDL 
Map.ts_inspector(
    left_ts=collection_nlcd, right_ts=collection_cdl, left_names=labels_nlcd, right_names=labels_cdl
)

# add legends using builtin_legends
Map.add_legend(title="NLCD Land Cover", builtin_legend='NLCD',position='bottomleft')
Map.add_legend(title="CDL Land Cover", builtin_legend='USDA/NASS/CDL', position='bottomright')
Map

Map(bottom=1851.0, center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=D…

## NLCD vs Landsat timeseries comaprison, Great Salt Lake, Utah

In [ ]:
# create new roi (Great Salt Lake) via polygon coordinates
roi = ee.Geometry.Polygon(
    [
        [
            [-113.5984664288,39.9405814735],
            [-111.0976558141,39.9405814735],
            [-111.0976558141,41.7803147911],
            [-113.5984664288,41.7803147911],
            [-113.5984664288,39.9405814735],
        ]
    ],
    None,
    False,
)

In [ ]:
# Pull NLCD images, select landcover, and clip to roi 
NLCD2001 = ee.Image('USGS/NLCD/NLCD2001').select('landcover').clip(roi)
NLCD2006 = ee.Image('USGS/NLCD/NLCD2006').select('landcover').clip(roi)
NLCD2011 = ee.Image('USGS/NLCD/NLCD2011').select('landcover').clip(roi)
NLCD2016 = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(roi)

In [ ]:
# Create image collect for NLCD across years and create labels for ts_inspector
NLCD = ee.ImageCollection([NLCD2001, NLCD2006, NLCD2011, NLCD2016])
NLCD_layer_names = ['NLCD ' + str(year) for year in range(2001, 2017, 5)]
print(NLCD_layer_names)

['NLCD 2001', 'NLCD 2006', 'NLCD 2011', 'NLCD 2016']


In [ ]:
# pull landsat timesereies for roi from 2001 to 2016
landsat_ts = geemap.landsat_timeseries(
    roi=roi, start_year=2001, end_year=2016, start_date='01-01', end_date='12-31'
)

In [ ]:
# Create labels landsat by year for ts_inspector
landsat_layer_names = ['Landsat ' + str(year) for year in range(2001, 2017)]

In [ ]:
# visualize landsat imagery using Near Infrared Reflectance (NIR) 
landsat_vis = {
    'min': 0,
    'max': 0.3,
    'gamma': [1, 1, 1],
    'bands': ['NIR', 'Red', 'Green'],
}
# vizualize nlcd using landcover band
nlcd_vis = {'bands': ['landcover']}

In [ ]:
Map = geemap.Map()
# map timeseries with landsat on left and nlcd on right
Map.ts_inspector(
    left_ts=landsat_ts,
    right_ts=NLCD,
    left_names=landsat_layer_names,
    right_names=NLCD_layer_names,
    left_vis=landsat_vis,
    right_vis=nlcd_vis,
)
Map.centerObject(roi, zoom=8)
Map.add_legend(title="NLCD Land Cover", builtin_legend='NLCD',position='topright')
Map

Map(center=[40.85844006932575, -112.34806112145009], controls=(WidgetControl(options=['position', 'transparent…